In [1]:
import tensorflow as tf
from nltk.corpus import words
import timeit
import geocoder
import time
import sys
import numpy as np
import pandas as pd
import binascii
import pickle
import textwrap

Saving the corpus into CSV

In [2]:
corpus = words.words()
df = pd.DataFrame(data={"Corpus": corpus})
df.to_csv("../datasets/corpus.csv", sep=',',index=False, header=True)

Creating bitcode to each word and Generating random language

In [3]:
def text_to_bits(text, encoding='utf-8', errors='surrogatepass'):
    bits = bin(int(binascii.hexlify(text.encode(encoding, errors)), 16))[2:]
    return bits.zfill(8 * ((len(bits) + 7) // 8))

def text_from_bits(bits, encoding='utf-8', errors='surrogatepass'):
    n = int(bits, 2)
    return int2bytes(n).decode(encoding, errors)

def int2bytes(i):
    hex_string = '%x' % i
    n = len(hex_string)
    return binascii.unhexlify(hex_string.zfill(n + (n & 1)))

In [14]:
bit_code = list()
encrypted = list()

for i in corpus:
    bits = text_to_bits(i)
    bit_code.append(bits)
    t = np.random.randint(2, size= len(bits))
    "".join(map(str, t))
    t = "".join([str(a) for a in t])
    encrypted.append(t)

In [19]:
df = pd.DataFrame(data={"input": bit_code, "output": encrypted})
df.to_csv("../datasets/discriminatorData.csv", sep=',',index=False, header=True)

In [36]:
bit = list()
prediction = list()

for i in bit_code:
    bit.append(i)
    prediction.append(1)
    
for i in encrypted:
    bit.append(i)
    prediction.append(0)

1 = english language ; 0 = machine generated language

In [39]:
df = pd.DataFrame(data={"bits": bit, "category": prediction})
df.to_csv("../datasets/training.csv", sep=',',index=False, header=True)

# ANN Dataset Creation

In [4]:
ann_dataset = list()
c = 0
for i in corpus:
    t = text_to_bits(i)
    l = len(t)
    k = 8 - (l%8)
    if k != 24:
        s = ''
        for j in range(k):
            s = s + "0"
        t = s + t
    
    s = textwrap.wrap(t,8)
    ann_dataset.append(s)

In [20]:
mylist = list( dict.fromkeys(final) )

In [28]:
import string
a = string.printable
ascii_list = list()
for i in a:
    ascii_list.append(text_to_bits(i))
encrypted = list()
c = 1
for i in ascii_list:
    t = np.random.randint(2, size= 8)
    "".join(map(str, t))
    t = "".join([str(a) for a in t])
    if t in encrypted or t in ascii_list:
        flag = 1
        while flag:
            t = np.random.randint(2, size= 8)
            "".join(map(str, t))
            t = "".join([str(a) for a in t])
            if t in encrypted or t in ascii_list:
                flag = 1
            else:
                encrypted.append(t)
                flag = 0
    else:
        encrypted.append(t)

In [52]:
df = pd.DataFrame(data={"input": ascii_list, "output": encrypted})
df.to_csv("../datasets/ascii.csv", sep=',',index=False, header=True)

In [6]:
import itertools
final = list(itertools.chain.from_iterable(ann_dataset))
next_row = list(itertools.chain.from_iterable(ann_dataset))

In [15]:
mylist = list( dict.fromkeys(final) )

In [19]:
for i in mylist:
    print(text_from_bits(i))
    


 

A

a

l

i

m

n

r

d

v

k

w

o

f

c

t

e

u

b

h

y

s

p

g

z

x

j

q

B

C

D

E

F

G

H

I

J

-

P

K

L

M

N

O

Q

R

S

T

U

V

W

X

Y

Z


In [134]:
a = next_row.pop(0)
next_row.append(a)

In [136]:
df = pd.DataFrame(data={"first word": final, "next word": next_row})
df.to_csv("../datasets/ann_dataset.csv", sep=',',index=False, header=True)

# Creating discriminator model

In [48]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier


df = pd.read_csv('../datasets/training.csv')
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,[0]], df.iloc[:,[1]], random_state=0)

In [41]:
clf = KNeighborsClassifier(n_neighbors=3)


In [42]:
clf.fit(X_train, y_train)

/home/cp/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """Entry point for launching an IPython kernel.


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

Saving model

In [ ]:
filename = '../models/discriminator.sav'
pickle.dump(clf, open(filename, 'wb'))